# Validation for Anisotropic SGS Model

In [1]:
import numpy as np
from numpy import linalg as la
import math as m
import matplotlib.pyplot as plt
import evtk
import vtk
from vtk.util import numpy_support as VN

/Users/rbalin/opt/miniconda3/envs/ssim/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
# Define class for the model
class SGS:
    def __init__(self, data_path, model_path, crd_path=None, device="cpu"):
        self.data_path = data_path
        self.crd_path = crd_path
        self.model_path = model_path
        self.device = device
        self.test_data = None
        self.X = None
        self.y = None
        self.SGS = None
        self.crd = None
        self.model = None
        self.y_pred_glob = None
        self.min_val = np.zeros(6)
        self.max_val = np.zeros(6)
        self.eigvecs_aligned = None
        self.SpO = None
        self.Deltaij_norm = None
        self.SGS_GM = None
        self.new_outputs = None

    # Load model
    def load_model(self):
        self.model = torch.jit.load(self.model_path+"_jit.pt", 
                                    map_location=torch.device(self.device))
        tmp = np.loadtxt(self.model_path+"_scaling.txt")
        self.min_val = tmp[:,0]
        self.max_val = tmp[:,1]
        
    # Load inputs and outputs from file directly
    def load_data(self):
        extension = self.data_path.split(".")[-1]
        #if "npy" in extension:
        #    test_data = np.load(self.data_path)
        #    self.y = test_data[:,:6]
        #    self.X = test_data[:,6:]
        #    self.crd = np.load(self.crd_path)
        if "vtu" in extension or "vtk" in extension:
            from vtk.util import numpy_support as VN
            reader = vtk.vtkXMLUnstructuredGridReader()
            reader.SetFileName(self.data_path)
            reader.Update()
            output = reader.GetOutput()
            self.X = np.hstack((VN.vtk_to_numpy(output.GetPointData().GetArray("input123")),
                                VN.vtk_to_numpy(output.GetPointData().GetArray("input456"))))
            self.y = np.hstack((VN.vtk_to_numpy(output.GetPointData().GetArray("output123")),
                                VN.vtk_to_numpy(output.GetPointData().GetArray("output456"))))
            self.SGS = np.hstack((VN.vtk_to_numpy(output.GetPointData().GetArray("SGS_diag")),
                                  VN.vtk_to_numpy(output.GetPointData().GetArray("SGS_offdiag"))))
            self.crd = VN.vtk_to_numpy(output.GetPoints().GetData())
        return output
    
    # Compute inputs and outputs from raw data
    def compute_data(self, scaling, alignment="vorticity"):
        extension = self.data_path.split(".")[-1]
        if "vtu" in extension or "vtk" in extension:
            reader = vtk.vtkXMLUnstructuredGridReader()
            reader.SetFileName(self.data_path)
            reader.Update()
            polydata = reader.GetOutput()
            self.SGS = np.hstack((VN.vtk_to_numpy(polydata.GetPointData().GetArray("SGS_diag")),
                                  VN.vtk_to_numpy(polydata.GetPointData().GetArray("SGS_offdiag"))))
            self.crd = VN.vtk_to_numpy(polydata.GetPoints().GetData())
            GradU = np.hstack((VN.vtk_to_numpy(polydata.GetPointData().GetArray("GradUFilt")),
                            VN.vtk_to_numpy(polydata.GetPointData().GetArray("GradVFilt")),
                            VN.vtk_to_numpy(polydata.GetPointData().GetArray("GradZFilt"))))
            GradU = np.reshape(GradU, (-1,3,3))
            Delta = VN.vtk_to_numpy(polydata.GetPointData().GetArray("gij"))
            
        nsamples = GradU.shape[0]
        self.eigvecs_aligned = np.zeros((nsamples,3,3))
        self.SpO = np.zeros((nsamples,))
        self.Deltaij_norm = np.zeros((nsamples,))
        self.y = np.zeros((nsamples,6))
        self.X = np.zeros((nsamples,6))
        Deltaij = np.zeros((3,3))
        Sij = np.zeros((3,3))
        Oij = np.zeros((3,3))
        vort = np.zeros((3,))
        vort_Sframe = np.zeros((3,))
        tmp = np.zeros((3,3))
        for i in range(nsamples):
            Deltaij[0,0] = Delta[i,0]*scaling[0]
            Deltaij[1,1] = Delta[i,1]*scaling[1]
            Deltaij[2,2] = Delta[i,2]*scaling[2]
            self.Deltaij_norm[i] = m.sqrt(Deltaij[0,0]**2 + Deltaij[1,1]**2 + Deltaij[2,2]**2)
            Deltaij = Deltaij / (self.Deltaij_norm[i]+1.0e-14)
            
            Gij = np.matmul(GradU[i],Deltaij)
            Sij[0,0] = Gij[0,0]
            Sij[1,1] = Gij[1,1]
            Sij[2,2] = Gij[2,2]
            Sij[0,1] = 0.5*(Gij[0,1]+Gij[1,0])
            Sij[0,2] = 0.5*(Gij[0,2]+Gij[2,0])
            Sij[1,2] = 0.5*(Gij[1,2]+Gij[2,1])
            Sij[1,0] = Sij[0,1]
            Sij[2,0] = Sij[0,2]
            Sij[2,1] = Sij[1,2]
            Oij[0,1] = 0.5*(Gij[0,1]-Gij[1,0])
            Oij[0,2] = 0.5*(Gij[0,2]-Gij[2,0])
            Oij[1,2] = 0.5*(Gij[1,2]-Gij[2,1])
            Oij[1,0] = -Oij[0,1]
            Oij[2,0] = -Oij[0,2]
            Oij[2,1] = -Oij[1,2]
            vort[0] = -2*Oij[1,2]
            vort[1] = -2*Oij[0,2]
            vort[2] = -2*Oij[0,1]
            Sij_norm = m.sqrt(Sij[0,0]**2+Sij[1,1]**2+Sij[2,2]**2 \
                          + 2*(Sij[0,1]**2+Sij[0,2]**2+Sij[1,2]**2))
            vort_norm = m.sqrt(vort[0]**2 + vort[1]**2 + vort[2]**2)
            self.SpO[i] = Sij_norm**2 + 0.5*vort_norm**2
            
            #evals, evecs = jacobi(Sij) 
            evals, evecs = la.eig(Sij)
            if (alignment=="vorticity"):
                vec = vort.copy()
            elif (alignment=="wall-normal"):
                vec = np.array([0,1,0])
            else:
                print("Alignment option not known, used default vorticity alignment")
                vec = vort.copy()
            lda, eigvecs, self.eigvecs_aligned[i] = self.align_tensors(evals,evecs,vec)
            
            vort_Sframe[0] = np.dot(vort,self.eigvecs_aligned[i,:,0])
            vort_Sframe[1] = np.dot(vort,self.eigvecs_aligned[i,:,1])
            vort_Sframe[2] = np.dot(vort,self.eigvecs_aligned[i,:,2])
            self.X[i,0] = lda[0] / (m.sqrt(self.SpO[i])+1.0e-14)
            self.X[i,1] = lda[1] / (m.sqrt(self.SpO[i])+1.0e-14)
            self.X[i,2] = lda[2] / (m.sqrt(self.SpO[i])+1.0e-14)
            self.X[i,3] = vort_Sframe[0] / (m.sqrt(self.SpO[i])+1.0e-14)
            self.X[i,4] = vort_Sframe[1] / (m.sqrt(self.SpO[i])+1.0e-14)
            self.X[i,5] = 1.25e-5 / (self.Deltaij_norm[i]**2 * m.sqrt(self.SpO[i]) + 1.0e-14)
        
            tmp[0,0] = self.SGS[i,0] / (self.Deltaij_norm[i]**2 * self.SpO[i] + 1.0e-14)
            tmp[1,1] = self.SGS[i,1] / (self.Deltaij_norm[i]**2 * self.SpO[i] + 1.0e-14)
            tmp[2,2] = self.SGS[i,2] / (self.Deltaij_norm[i]**2 * self.SpO[i] + 1.0e-14)
            tmp[0,1] = self.SGS[i,3] / (self.Deltaij_norm[i]**2 * self.SpO[i] + 1.0e-14)
            tmp[0,2] = self.SGS[i,4] / (self.Deltaij_norm[i]**2 * self.SpO[i] + 1.0e-14)
            tmp[1,2] = self.SGS[i,5] / (self.Deltaij_norm[i]**2 * self.SpO[i] + 1.0e-14)
            tmp[1,0] = tmp[0,1]
            tmp[2,0] = tmp[0,2]
            tmp[2,1] = tmp[1,2]
            tmp = np.matmul(np.transpose(self.eigvecs_aligned[i]),
                           np.matmul(tmp,self.eigvecs_aligned[i]))
            self.y[i,0] = tmp[0,0]
            self.y[i,1] = tmp[1,1]
            self.y[i,2] = tmp[2,2]
            self.y[i,3] = tmp[0,1]
            self.y[i,4] = tmp[0,2]
            self.y[i,5] = tmp[1,2]
            
        return polydata
        
    # Run inference on all data for global metrics
    def test_global(self, X_test=None, y_test=None):
        if X_test and y_test:
            X = X_test; y = y_test
        else:
            X = self.X; y = self.y; SGS = self.SGS
        X_torch = torch.from_numpy(np.float32(X))
        self.y_pred_glob = self.model(X_torch).detach().numpy()
        self.y_pred_glob = self.undo_min_max(self.y_pred_glob)
        mse_glob, mse_channel = self.MSE(y,self.y_pred_glob)
        cc_glob, cc_channel = self.CC(y,self.y_pred_glob)
        print("Inference on global data:")
        print(f"Output global MSE = {mse_glob:>8e}")
        print(f"Output channel MSE = {mse_channel[0]:>8e} {mse_channel[1]:>8e} {mse_channel[2]:>8e} {mse_channel[3]:>8e} {mse_channel[4]:>8e} {mse_channel[5]:>8e}")
        print(f"Output global Corr. Coefficient = {cc_glob:>8e}")
        print(f"Output channel Corr. Coefficient = {cc_channel[0]:>8e} {cc_channel[1]:>8e} {cc_channel[2]:>8e} {cc_channel[3]:>8e} {cc_channel[4]:>8e} {cc_channel[5]:>8e}")
        self.SGS_pred_glob = self.compute_SGS(self.y_pred_glob)
        mse_glob, mse_channel = self.MSE(SGS,self.SGS_pred_glob)
        rrmse_glob, rrmse_channel = self.RRMSE(SGS,self.SGS_pred_glob)
        cc_glob, cc_channel = self.CC(SGS,self.SGS_pred_glob)
        print(f"SGS global MSE = {mse_glob:>8e}")
        print(f"SGS channel MSE = {mse_channel[0]:>8e} {mse_channel[1]:>8e} {mse_channel[2]:>8e} {mse_channel[3]:>8e} {mse_channel[4]:>8e} {mse_channel[5]:>8e}")
        print(f"SGS global RRMSE = {rrmse_glob:>8e}")
        print(f"SGS channel RRMSE = {rrmse_channel[0]:>8e} {rrmse_channel[1]:>8e} {rrmse_channel[2]:>8e} {rrmse_channel[3]:>8e} {rrmse_channel[4]:>8e} {rrmse_channel[5]:>8e}")
        print(f"SGS global Corr. Coefficient = {cc_glob:>8e}")
        print(f"SGS channel Corr. Coefficient = {cc_channel[0]:>8e} {cc_channel[1]:>8e} {cc_channel[2]:>8e} {cc_channel[3]:>8e} {cc_channel[4]:>8e} {cc_channel[5]:>8e}")
        print("")
    
    # Run inference one 1 wall-parallel layer at a time
    def test_y_layers(self, X_test=None, y_test=None, crd_test=None):
        if X_test and y_test and crd_test:
            X = X_test; y = y_test; crd = crd_test
        else:
            X = self.X; y = self.y; crd = self.crd
        crd_y = np.unique(crd[:,1])
        ny = crd_y.size
        mse_y = np.zeros_like(crd_y)
        cc_y = np.zeros_like(crd_y)
        for j in range(ny):
            index = np.where(crd[:,1]==crd_y[j])
            X_tmp = torch.from_numpy(np.float32(X[index]))
            y_pred_tmp = self.model(X_tmp).detach().numpy()
            y_pred_tmp = self.undo_min_max(y_pred_tmp)
            mse_y[j] = self.MSE(y[index],y_pred_tmp)
            cc_y[j] = self.CC(y[index],y_pred_tmp)
        return crd_y, mse_y, cc_y

    # Undo the model's min-max scaling
    def undo_min_max(self,y):
        for i in range(6):
            y[:,i] = y[:,i] * (self.max_val[i] - self.min_val[i]) + self.min_val[i]
        return y

    # Deine MSE function
    def MSE(self, y, y_pred):
        glob_mse = np.square(y-y_pred).mean()
        channel_mse = []
        for i in range(6):
            tmp = np.square(y[:,i]-y_pred[:,i]).mean()
            channel_mse.append(tmp)
        return glob_mse, channel_mse
    
    # Define Relative Root MSE function
    def RRMSE(self,y,y_pred):
        glob_rrmse = m.sqrt(np.square(y-y_pred).mean())/np.std(y)
        channel_rrmse = []
        for i in range(6):
            tmp = m.sqrt(np.square(y[:,i]-y_pred[:,i]).mean())/np.std(y[:,i])
            channel_rrmse.append(tmp)
        return glob_rrmse, channel_rrmse
    
    # Define Correlation Coefficient function
    def CC(self, y, y_pred):
        glob_cc = np.corrcoef([np.ndarray.flatten(y),np.ndarray.flatten(y_pred)])[0][1]
        channel_cc = []
        for i in range(6):
            tmp = np.corrcoef([y[:,i],y_pred[:,i]])[0][1]
            channel_cc.append(tmp)
        return glob_cc, channel_cc

    # Align the eigenvalues and eignevectors according to the local vorticity
    def align_tensors(self,evals,evecs,vec):
        if (evals[0]<1.0e-8 and evals[1]<1.0e-8 and evals[2]<1.0e-8):
            index = [0,1,2]
        else:
            index = np.flip(np.argsort(evals))
        lda = evals[index]
        vec_norm = m.sqrt(vec[0]**2 + vec[1]**2 + vec[2]**2)
        vec = vec/vec_norm
        eigvec = np.zeros((3,3))
        eigvec[:,0] = evecs[:,index[0]]
        eigvec[:,1] = evecs[:,index[1]]
        eigvec[:,2] = evecs[:,index[2]]
        eigvec_vort_aligned = eigvec.copy()
        if (np.dot(vec,eigvec_vort_aligned[:,0]) < np.dot(vec,-eigvec_vort_aligned[:,0])):
            eigvec_vort_aligned[:,0] = -eigvec_vort_aligned[:,0]
        if (np.dot(vec,eigvec_vort_aligned[:,2]) < np.dot(vec,-eigvec_vort_aligned[:,2])):
            eigvec_vort_aligned[:,2] = -eigvec_vort_aligned[:,2]
        eigvec_vort_aligned[0,1] = (eigvec_vort_aligned[1,2]*eigvec_vort_aligned[2,0]) \
                               - (eigvec_vort_aligned[2,2]*eigvec_vort_aligned[1,0])
        eigvec_vort_aligned[1,1] = (eigvec_vort_aligned[2,2]*eigvec_vort_aligned[0,0]) \
                               - (eigvec_vort_aligned[0,2]*eigvec_vort_aligned[2,0])
        eigvec_vort_aligned[2,1] = (eigvec_vort_aligned[0,2]*eigvec_vort_aligned[1,0]) \
                               - (eigvec_vort_aligned[1,2]*eigvec_vort_aligned[0,0])
        return lda, eigvec, eigvec_vort_aligned

    # Compute the transformation needed to obtain the physical stresses
    def compute_transformation(self, polydata, scaling, alignment="vorticity"):
        GradU = np.hstack((VN.vtk_to_numpy(polydata.GetPointData().GetArray("GradUFilt")),
                            VN.vtk_to_numpy(polydata.GetPointData().GetArray("GradVFilt")),
                            VN.vtk_to_numpy(polydata.GetPointData().GetArray("GradZFilt"))))
        GradU = np.reshape(GradU, (-1,3,3))
        Delta = VN.vtk_to_numpy(polydata.GetPointData().GetArray("gij"))
        nsamples = GradU.shape[0]
        self.eigvecs_aligned = np.zeros((nsamples,3,3))
        self.SpO = np.zeros((nsamples,))
        self.Deltaij_norm = np.zeros((nsamples,))
        Deltaij = np.zeros((3,3))
        Sij = np.zeros((3,3))
        Oij = np.zeros((3,3))
        vort = np.zeros((3,))
        for i in range(nsamples):
            Deltaij[0,0] = Delta[i,0]*scaling[0]
            Deltaij[1,1] = Delta[i,1]*scaling[1]
            Deltaij[2,2] = Delta[i,2]*scaling[2]
            self.Deltaij_norm[i] = m.sqrt(Deltaij[0,0]**2 + Deltaij[1,1]**2 + Deltaij[2,2]**2)
            Deltaij = Deltaij / (self.Deltaij_norm[i]+1.0e-14)
            Gij = np.matmul(GradU[i],Deltaij)
            Sij[0,0] = Gij[0,0]
            Sij[1,1] = Gij[1,1]
            Sij[2,2] = Gij[2,2]
            Sij[0,1] = 0.5*(Gij[0,1]+Gij[1,0])
            Sij[0,2] = 0.5*(Gij[0,2]+Gij[2,0])
            Sij[1,2] = 0.5*(Gij[1,2]+Gij[2,1])
            Sij[1,0] = Sij[0,1]
            Sij[2,0] = Sij[0,2]
            Sij[2,1] = Sij[1,2]
            Oij[0,1] = 0.5*(Gij[0,1]-Gij[1,0])
            Oij[0,2] = 0.5*(Gij[0,2]-Gij[2,0])
            Oij[1,2] = 0.5*(Gij[1,2]-Gij[2,1])
            Oij[1,0] = -Oij[0,1]
            Oij[2,0] = -Oij[0,2]
            Oij[2,1] = -Oij[1,2]
            vort[0] = -2*Oij[1,2]
            vort[1] = -2*Oij[0,2]
            vort[2] = -2*Oij[0,1]
            Sij_norm = m.sqrt(Sij[0,0]**2+Sij[1,1]**2+Sij[2,2]**2 \
                          + 2*(Sij[0,1]**2+Sij[0,2]**2+Sij[1,2]**2))
            vort_norm = m.sqrt(vort[0]**2 + vort[1]**2 + vort[2]**2)
            self.SpO[i] = Sij_norm**2 + 0.5*vort_norm**2
            #evals, evecs = jacobi(Sij) 
            evals, evecs = la.eig(Sij)
            if (alignment=="vorticity"):
                vec = vort.copy()
            elif (alignment=="wall-normal"):
                vec = np.array([0,1,0])
            else:
                print("Alignment option not known, used default vorticity alignment")
                vec = vort.copy()
            lda, eigvecs, self.eigvecs_aligned[i] = self.align_tensors(evals,evecs,vec)

    # Compute physical stresses
    def compute_SGS(self,y,index=None):
        tmp = np.zeros((3,3))
        nsamples = y.shape[0]
        SGS_pred = np.zeros((nsamples,6))
        for i in range(nsamples):
            tmp[0,0] = y[i,0]
            tmp[1,1] = y[i,1]
            tmp[2,2] = y[i,2]
            tmp[0,1] = y[i,3]
            tmp[0,2] = y[i,4]
            tmp[1,2] = y[i,5]
            tmp[1,0] = tmp[0,1]
            tmp[2,0] = tmp[0,2]
            tmp[2,1] = tmp[1,2]
            tmp = np.matmul(self.eigvecs_aligned[i],tmp)
            tmp = np.matmul(tmp,np.transpose(self.eigvecs_aligned[i]))
            SGS_pred[i,0] = tmp[0,0] * (self.Deltaij_norm[i]**2 * self.SpO[i])
            SGS_pred[i,1] = tmp[1,1] * (self.Deltaij_norm[i]**2 * self.SpO[i])
            SGS_pred[i,2] = tmp[2,2] * (self.Deltaij_norm[i]**2 * self.SpO[i])
            SGS_pred[i,3] = tmp[0,1] * (self.Deltaij_norm[i]**2 * self.SpO[i])
            SGS_pred[i,4] = tmp[0,2] * (self.Deltaij_norm[i]**2 * self.SpO[i])
            SGS_pred[i,5] = tmp[1,2] * (self.Deltaij_norm[i]**2 * self.SpO[i])
        return SGS_pred

    # Compute the stress with the Gradient Model
    def gradient_SGS_model(self, polydata, scaling):
        GradU = np.hstack((VN.vtk_to_numpy(polydata.GetPointData().GetArray("GradUFilt")),
                            VN.vtk_to_numpy(polydata.GetPointData().GetArray("GradVFilt")),
                            VN.vtk_to_numpy(polydata.GetPointData().GetArray("GradZFilt"))))
        GradU = np.reshape(GradU, (-1,3,3))
        Delta = VN.vtk_to_numpy(polydata.GetPointData().GetArray("gij"))
        Delta[:,0] = Delta[:,0]*scaling[0]
        Delta[:,1] = Delta[:,1]*scaling[1]
        Delta[:,2] = Delta[:,2]*scaling[2]
        nsamples = GradU.shape[0]
        self.SGS_GM = np.zeros((nsamples,6))
        for i in range(nsamples):
            self.SGS_GM[i,0] = (Delta[i,0]**2 * GradU[i,0,0]**2 + \
                                Delta[i,1]**2 * GradU[i,0,1]**2 + \
                                Delta[i,2]**2 * GradU[i,0,2]**2) / 12 # 11
            self.SGS_GM[i,1] = (Delta[i,0]**2 * GradU[i,1,0]**2 + \
                                Delta[i,1]**2 * GradU[i,1,1]**2 + \
                                Delta[i,2]**2 * GradU[i,1,2]**2) / 12 # 22
            self.SGS_GM[i,2] = (Delta[i,0]**2 * GradU[i,2,0]**2 + \
                                Delta[i,1]**2 * GradU[i,2,1]**2 + \
                                Delta[i,2]**2 * GradU[i,2,2]**2) / 12 # 33
            self.SGS_GM[i,3] = (Delta[i,0]**2 * GradU[i,0,0]*GradU[i,1,0] + \
                                Delta[i,1]**2 * GradU[i,0,1]*GradU[i,1,1] + \
                                Delta[i,2]**2 * GradU[i,0,2]*GradU[i,1,2]) / 12 # 12
            self.SGS_GM[i,4] = (Delta[i,0]**2 * GradU[i,0,0]*GradU[i,2,0] + \
                                Delta[i,1]**2 * GradU[i,0,1]*GradU[i,2,1] + \
                                Delta[i,2]**2 * GradU[i,0,2]*GradU[i,2,2]) / 12 # 13
            self.SGS_GM[i,5] = (Delta[i,0]**2 * GradU[i,1,0]*GradU[i,2,0] + \
                                Delta[i,1]**2 * GradU[i,1,1]*GradU[i,2,1] + \
                                Delta[i,2]**2 * GradU[i,1,2]*GradU[i,2,2]) / 12 # 23
        print("Gradient Model on global data:")
        mse_glob, mse_channel = self.MSE(self.SGS,self.SGS_GM)
        rrmse_glob, rrmse_channel = self.RRMSE(self.SGS,self.SGS_GM)
        cc_glob, cc_channel = self.CC(self.SGS,self.SGS_GM)
        print(f"SGS global MSE = {mse_glob:>8e}")
        print(f"SGS channel MSE = {mse_channel[0]:>8e} {mse_channel[1]:>8e} {mse_channel[2]:>8e} {mse_channel[3]:>8e} {mse_channel[4]:>8e} {mse_channel[5]:>8e}")
        print(f"SGS global RRMSE = {rrmse_glob:>8e}")
        print(f"SGS channel RRMSE = {rrmse_channel[0]:>8e} {rrmse_channel[1]:>8e} {rrmse_channel[2]:>8e} {rrmse_channel[3]:>8e} {rrmse_channel[4]:>8e} {rrmse_channel[5]:>8e}")
        print(f"SGS global Corr. Coefficient = {cc_glob:>8e}")
        print(f"SGS channel Corr. Coefficient = {cc_channel[0]:>8e} {cc_channel[1]:>8e} {cc_channel[2]:>8e} {cc_channel[3]:>8e} {cc_channel[4]:>8e} {cc_channel[5]:>8e}")
        print("")
        
    # Save to vtk files for import into Paraview
    def save_vtk(self, polydata, fname):
        from vtk.numpy_interface import dataset_adapter as dsa
        new = dsa.WrapDataObject(polydata)
        new.PointData.append(self.X[:,:3], "X123")
        new.PointData.append(self.X[:,3:], "X456")
        new.PointData.append(self.y[:,:3], "y123")
        new.PointData.append(self.y[:,3:], "y456")
        new.PointData.append(self.y_pred_glob[:,:3], "pred_output123")
        new.PointData.append(self.y_pred_glob[:,3:], "pred_output456")
        new.PointData.append(self.SGS_pred_glob[:,:3], "pred_SGS_diag")
        new.PointData.append(self.SGS_pred_glob[:,3:], "pred_SGS_offdiag")
        if (self.SGS_GM is not None):
            new.PointData.append(self.SGS_GM[:,:3], "SGSGM_diag")
            new.PointData.append(self.SGS_GM[:,3:], "SGSGM_offdiag")
        writer = vtk.vtkXMLUnstructuredGridWriter()
        writer.SetFileName(fname)
        writer.SetInputData(new.VTKObject)
        writer.Write()

In [26]:
base_path = "/Users/rbalin/Documents/Research/ALCF_PostDoc/Conferences/PASC23/FlatPlate/Train/CRS_6-15_4d/"
data_path = base_path+"train_data_noDamp/FlatPlate_ReTheta1000_6-15_ts30005_3x_noDamp_jacobi_test.vtu"
model_path = base_path+"models/3x/NNmodel"
base = SGS(data_path, model_path)
base.load_model()
polydata = base.compute_data([3, 3, 3])
base.test_global()
base.gradient_SGS_model(polydata,[3, 3, 3])
#crd_y, mse_y_off_bl_3x_3x, cc_y_off_bl_3x_3x =  off_bl_3x.test_y_layers()
base.save_vtk(polydata,model_path+"_predictions.vtu")

Inference on global data:
Output global MSE = 3.246897e-03
Output channel MSE = 3.539688e-03 3.957756e-03 4.368311e-03 1.646567e-03 4.346861e-03 1.622199e-03
Output global Corr. Coefficient = 7.180666e-01
Output channel Corr. Coefficient = 4.569402e-01 3.675810e-01 4.544712e-01 2.826716e-01 4.527419e-01 2.180301e-01
SGS global MSE = 2.621438e-07
SGS channel MSE = 4.917885e-07 1.691225e-07 6.116621e-07 7.439487e-08 1.714260e-07 5.446864e-08
SGS global RRMSE = 7.020776e-01
SGS channel RRMSE = 4.868009e-01 1.102578e+00 1.621267e+00 1.037689e+00 1.180867e+00 1.143442e+00
SGS global Corr. Coefficient = 7.523834e-01
SGS channel Corr. Coefficient = 8.783445e-01 6.715417e-01 5.439260e-01 3.360099e-01 1.818338e-01 1.730715e-01

Gradient Model on global data:
SGS global MSE = 2.296204e-07
SGS channel MSE = 1.054173e-06 8.480483e-08 1.467987e-07 2.963314e-08 4.429739e-08 1.801583e-08
SGS global RRMSE = 6.570836e-01
SGS channel RRMSE = 7.127186e-01 7.807624e-01 7.942547e-01 6.549144e-01 6.002770e-

In [23]:
base_path = "/Users/rbalin/Documents/Research/ALCF_PostDoc/Conferences/PASC23/FlatPlate/Train/CRS_6-15_4d/"
data_path = base_path+"train_data_noDamp/FlatPlate_ReTheta1000_6-15_ts30005_3x_noDamp_jacobi_test.vtu"
model_path = base_path+"models/3x/py_inputs/NNmodel_w3"
w3 = SGS(data_path, model_path)
w3.load_model()
polydata = w3.compute_data([3, 3, 3],alignment="vorticity")
w3.test_global()
w3.gradient_SGS_model(polydata,[3, 3, 3])
#crd_y, mse_y_off_bl_3x_3x, cc_y_off_bl_3x_3x =  off_bl_3x.test_y_layers()
w3.save_vtk(polydata,model_path+"_predictions.vtu")

Inference on global data:
Output global MSE = 4.221316e-03
Output channel MSE = 4.001724e-03 5.203323e-03 5.225553e-03 1.889396e-03 7.090880e-03 1.917023e-03
Output global Corr. Coefficient = 6.204601e-01
Output channel Corr. Coefficient = 3.769894e-01 9.623976e-02 3.140778e-01 1.146019e-01 -5.219536e-02 1.017159e-01
SGS global MSE = 3.477374e-07
SGS channel MSE = 5.948021e-07 1.856329e-07 9.509685e-07 6.924070e-08 2.089736e-07 7.680657e-08
SGS global Corr. Coefficient = 6.883615e-01
SGS channel Corr. Coefficient = 8.534257e-01 6.516297e-01 4.601485e-01 4.309426e-01 2.262061e-01 1.468678e-01

Gradient Model on global data:
SGS global MSE = 2.296204e-07
SGS channel MSE = 1.054173e-06 8.480483e-08 1.467987e-07 2.963314e-08 4.429739e-08 1.801583e-08
SGS global Corr. Coefficient = 9.306499e-01
SGS channel Corr. Coefficient = 9.340462e-01 8.876476e-01 8.913968e-01 8.744161e-01 8.947438e-01 8.520573e-01



In [24]:
base_path = "/Users/rbalin/Documents/Research/ALCF_PostDoc/Conferences/PASC23/FlatPlate/Train/CRS_6-15_4d/"
data_path = base_path+"train_data_noDamp/FlatPlate_ReTheta1000_6-15_ts30005_3x_noDamp_jacobi_test.vtu"
model_path = "/Users/rbalin/git-balin/data_driven_SGS_modeling/models/HIT/NNmodel_HIT"
HIT = SGS(data_path, model_path)
HIT.load_model()
polydata = HIT.compute_data([3, 3, 3],alignment="vorticity")
HIT.test_global()
HIT.gradient_SGS_model(polydata,[3, 3, 3])
#crd_y, mse_y_off_bl_3x_3x, cc_y_off_bl_3x_3x =  off_bl_3x.test_y_layers()
HIT.save_vtk(polydata,model_path+"_predictions.vtu")

Inference on global data:
Output global MSE = 6.030977e-03
Output channel MSE = 9.944710e-03 4.989023e-03 1.165483e-02 1.845985e-03 6.090003e-03 1.661316e-03
Output global Corr. Coefficient = 4.703511e-01
Output channel Corr. Coefficient = 1.517871e-01 2.451922e-01 2.665187e-01 8.829511e-02 5.499182e-02 1.690079e-01
SGS global MSE = 4.133878e-07
SGS channel MSE = 1.463806e-06 1.067721e-07 5.532002e-07 6.287522e-08 2.443976e-07 4.927575e-08
SGS global Corr. Coefficient = 5.000738e-01
SGS channel Corr. Coefficient = 7.179098e-01 6.547600e-01 3.988849e-01 4.287189e-01 -2.928798e-01 4.764092e-02

Gradient Model on global data:
SGS global MSE = 2.296204e-07
SGS channel MSE = 1.054173e-06 8.480483e-08 1.467987e-07 2.963314e-08 4.429739e-08 1.801583e-08
SGS global Corr. Coefficient = 9.306499e-01
SGS channel Corr. Coefficient = 9.340462e-01 8.876476e-01 8.913968e-01 8.744161e-01 8.947438e-01 8.520573e-01



In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
axs[0].plot(crd_y, mse_y_off_bl_3x_3x, 's', label="off_bl_3x_3x")
#axs[0].plot(crd_y, mse_y_off_bl_6x_6x, 'o', label="off_bl_6x_6x")
#axs[0].plot(crd_y, mse_y_off_bl_3x_6x, 'o', label="off_bl_3x_6x")
axs[1].plot(crd_y, cc_y_off_bl_3x_3x, 's', label="off_bl_3x")
#axs[1].plot(crd_y, cc_y_off_bl_6x_6x, 'o', label="off_bl_6x")
axs[0].set_yscale("log")
axs[0].set_xscale("log")
#axs[1].set_yscale("log")
axs[1].set_xscale("log")
axs[0].grid()
axs[1].grid()
fig.tight_layout(pad=3.0)
axs[0].set_ylabel('MSE')
axs[0].set_xlabel('y')
axs[0].set_title('Mean Squared Error')
axs[0].legend()
axs[1].set_ylabel('Correlation Coefficient')
axs[1].set_xlabel('y')
axs[1].set_title('Correlation Coefficient')
axs[1].legend()
#plt.savefig(fig_name+"_yerrors.png", dpi='figure', format="png")
plt.show()